In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from utils.model_util import create_model_and_diffusion
from eval.eval_humanml import load_dataset
import argparse
from data_loaders.humanml.scripts.motion_process import recover_from_ric
import torch
from sample.condition import cond_fn_key_location, get_target_and_inpt_from_kframes_batch
import copy
from functools import partial

/home/epinyoan/miniconda3/envs/gmd/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from utils.parser_util import eval_args
# sys.argv = ['/home/epinyoan/git/guided-motion-diffusion/eval/eval_humanml_condition.py', 
#             '--model_path', 
#             './save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224/model000500000.pt']


### 1. args_traj
# traj_model_path = "./save/traj_unet_adazero_swxs_eps_abs_fp16_clipwd_224/model000062500.pt"
# args_traj = eval_args(model_path=traj_model_path)


### 2. args
# args = eval_args()
# args.batch_size = 32 # This must be 32! Don't change it! otherwise it will cause a bug in R precision calc!
# args.num_frames = 196 # This must be 196!
# args.gen_two_stages = True
# skip_first_stage = False

In [4]:
# model_path is hard coded: "./save/traj_unet_adazero_swxs_eps_abs_fp16_clipwd_224/model000062500.pt"
import argparse
args = argparse.Namespace(abs_3d=True, action_file='', action_name='', adam_beta2=0.999, apply_zero_mask=False, arch='unet', augment_type='none', avg_model_beta=0, batch_size=32, classifier_scale=100.0, clip_range=6.0, cond_mask_prob=0.1, cuda=True, data_dir='', dataset='humanml', device=0, diffusion_steps=1000, dim_mults=[2, 2, 2, 2], do_inpaint=False, drop_redundant=False, emb_trans_dec=False, eval_batch_size=32, eval_during_training=False, eval_mode='wo_mm', eval_num_samples=1000, eval_rep_times=3, eval_split='test', eval_use_avg=True, ff_size=1024, full_traj_inpaint=False, gen_mse_loss=True, gen_reward_model=False, gen_two_stages=True, grad_clip=0, guidance_mode='no', guidance_param=2.5, impute_until=None, input_text='', interactive=False, lambda_fc=0.0, lambda_rcxyz=0.0, lambda_vel=0.0, latent_dim=512, layers=8, log_interval=1000, lr=0.0001, lr_anneal_steps=0, model_path='./save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224/model000500000.pt', motion_length=11.2, motion_length_cut=6.0, noise_schedule='cosine', num_frames=196, num_repetitions=3, num_samples=10, num_steps=600000, out_mult=16, output_dir='', overwrite=False, p2p_impute=True, predict_xstart=True, random_proj_scale=10.0, resume_checkpoint='', save_dir='save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224', save_interval=50000, seed=10, sigma_small=True, skip_first=None, std_scale_shift=(1.0, 0.0), text_prompt='', time_weighted_loss=False, train_platform_type='TensorboardPlatform', train_x0_as_eps=False, traj_extra_weight=1.0, traj_only=False, unconstrained=False, unet_adagn=True, unet_zero=True, use_ddim=False, use_fp16=False, use_random_proj=True, weight_decay=0.01, xz_only=False)
args_traj = argparse.Namespace(abs_3d=True, action_file='', action_name='', adam_beta2=0.999, apply_zero_mask=False, arch='unet', augment_type='none', avg_model_beta=0, batch_size=512, classifier_scale=100.0, clip_range=6.0, cond_mask_prob=0.1, cuda=True, data_dir='', dataset='humanml', device=0, diffusion_steps=1000, dim_mults=[0.125, 0.25, 0.5], do_inpaint=False, drop_redundant=False, emb_trans_dec=False, eval_batch_size=32, eval_during_training=False, eval_mode='wo_mm', eval_num_samples=1000, eval_rep_times=3, eval_split='test', eval_use_avg=True, ff_size=1024, full_traj_inpaint=False, gen_mse_loss=True, gen_reward_model=False, gen_two_stages=False, grad_clip=0, guidance_mode='no', guidance_param=2.5, impute_until=None, input_text='', interactive=False, lambda_fc=0.0, lambda_rcxyz=0.0, lambda_vel=0.0, latent_dim=512, layers=8, log_interval=125, lr=0.0001, lr_anneal_steps=0, model_path='./save/traj_unet_adazero_swxs_eps_abs_fp16_clipwd_224/model000062500.pt', motion_length=11.2, motion_length_cut=6.0, noise_schedule='cosine', num_frames=60, num_repetitions=3, num_samples=10, num_steps=112500, out_mult=16, output_dir='', overwrite=False, p2p_impute=True, predict_xstart=False, random_proj_scale=10.0, resume_checkpoint='', save_dir='save/traj_unet_adazero_swxs_eps_abs_fp16_clipwd_224', save_interval=6250, seed=10, sigma_small=True, skip_first=None, std_scale_shift=(1.0, 0.0), text_prompt='', time_weighted_loss=False, train_platform_type='TensorboardPlatform', train_x0_as_eps=False, traj_extra_weight=1.0, traj_only=True, unconstrained=False, unet_adagn=True, unet_zero=True, use_ddim=False, use_fp16=False, use_random_proj=False, weight_decay=0.01, xz_only=False)

In [5]:
gen_loader = load_dataset(args, args.num_frames, 'test', hml_mode='eval')
traj_model, traj_diffusion = create_model_and_diffusion(args_traj, gen_loader)
model_device = 'cuda'
traj_model.to(model_device)

Reading ././dataset/humanml_opt.txt
Loading dataset t2m ...
proj_matrix_dir = ./dataset
mode = eval
t2m dataset aug: none std_scale_shift: (1.0, 0.0)
t2m dataset drop redundant information: False


  0%|          | 0/4384 [00:00<?, ?it/s]

100%|██████████| 4384/4384 [00:02<00:00, 1815.06it/s]


Pointer Pointing at 0
Loading random projection matrix from ./dataset
nfeats: 1
Using UNET with lantent dim:  512  and mults:  [0.125, 0.25, 0.5]
dims:  [4, 64, 128, 256] mults:  [0.125, 0.25, 0.5]
[ models/temporal ] Channel dimensions: [(4, 64), (64, 128), (128, 256)]
EMBED TEXT
Loading CLIP...


In [19]:
motion, model_kwargs = next(iter(gen_loader))
gt_poses = motion.permute(0, 2, 3, 1)
gt_poses = gt_poses * gen_loader.dataset.std + gen_loader.dataset.mean  # [bs, 1, 196, 263]
# (x,y,z) [bs, 1, 120, njoints=22, nfeat=3]
gt_skel_motions = recover_from_ric(gt_poses.float(), 22, abs_3d=False)
gt_skel_motions = gt_skel_motions.view(-1, *gt_skel_motions.shape[2:]).permute(0, 2, 3, 1)
gt_skel_motions = traj_model.rot2xyz(x=gt_skel_motions, mask=None, pose_rep='xyz', glob=True, translation=True, 
                                    jointstype='smpl', vertstrans=True, betas=None, beta=0, glob_rot=None, get_rotations_back=False)

n_keyframe = 5
sampled_keyframes = torch.rand(motion.shape[0], n_keyframe) * model_kwargs['y']['lengths'].unsqueeze(-1)
sampled_keyframes = torch.floor(sampled_keyframes).int().sort()[0]
(target, target_mask, 
inpaint_traj, inpaint_traj_mask,
inpaint_traj_points, inpaint_traj_mask_points,
inpaint_motion, inpaint_mask, 
inpaint_motion_points, inpaint_mask_points) = \
    get_target_and_inpt_from_kframes_batch(gt_skel_motions, 
                                           sampled_keyframes, 
                                           gen_loader.dataset)
target = target.to(model_device)
target_mask = target_mask.to(model_device)

In [15]:
model_kwargs['y']['scale'] = torch.ones(motion.shape[0], device=model_device) * 2.5

In [16]:
traj_model_kwargs = copy.deepcopy(model_kwargs)
traj_model_kwargs['y']['traj_model'] = True
traj_model_kwargs['y']['inpainted_motion'] = inpaint_traj.to(model_device) # init_motion.to(model_device)
traj_model_kwargs['y']['inpainting_mask'] = inpaint_traj_mask.to(model_device)

impute_slack = 20
impute_until = 100
# Set when to stop imputing
traj_model_kwargs['y']['cond_until'] = impute_slack
traj_model_kwargs['y']['impute_until'] = impute_until
# NOTE: We have the option of switching the target motion from line to just key locations
# We call this a 'second stage', which will start after t reach 'impute_until'
traj_model_kwargs['y']['impute_until_second_stage'] = impute_slack
traj_model_kwargs['y']['inpainted_motion_second_stage'] = inpaint_traj_points.to(model_device)
traj_model_kwargs['y']['inpainting_mask_second_stage'] = inpaint_traj_mask_points.to(model_device)

In [20]:
sample_fn_traj = traj_diffusion.ddim_sample_loop
sample_traj = sample_fn_traj(
    traj_model,
    inpaint_traj.shape,
    clip_denoised=True,
    model_kwargs=traj_model_kwargs,  # <-- traj_kwards
    skip_timesteps=0,  # NOTE: for debugging, start from 900
    init_image=None,
    progress=True,
    dump_steps=None,
    noise=None,
    const_noise=False,
    cond_fn=partial(
        cond_fn_key_location, # cond_fn_sdf, #,
        transform=gen_loader.dataset.t2m_dataset.transform_th,
        inv_transform=gen_loader.dataset.t2m_dataset.inv_transform_th,
        target=target,
        target_mask=target_mask,
        kframes=[],
        abs_3d=True, # <<-- hard code,
        classifiler_scale=100.0,
        use_mse_loss=False),  # <<-- hard code
)   

  0%|          | 0/1000 [00:00<?, ?it/s]

 10%|█         | 102/1000 [00:05<00:49, 18.13it/s]

900: 3.341138


 20%|██        | 202/1000 [00:11<00:45, 17.70it/s]

800: 3.341138


...

200: 3.341138


 90%|█████████ | 902/1000 [00:50<00:05, 17.90it/s]

100: 3.341138


100%|██████████| 1000/1000 [00:55<00:00, 17.98it/s]

000: 3.341138


In [ ]:
sample_traj